<a href="https://colab.research.google.com/github/qoyyimil/pba/blob/main/notebooks/1_Scraping_Moovit_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Scraping Google Play Store Reviews - Moovit (com.tranzmate)
Tugas 1A - Pemodelan Bahasa Alami

In [1]:
!pip install google_play_scraper
!pip install textblob
!pip install seaborn
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=be0faea8248cea9bd6c062efec3091fb92bf80ffa343cba7803e9ecd0f341820
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [2]:
from google_play_scraper import app
import pandas as pd
import numpy as np
import sklearn
import requests
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import seaborn as sns
import textblob
#from wordcloud import WordCloud
from pathlib import Path
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report, accuracy_score

import pickle
import re
import time
import datetime                              # access to %%time, for timing individual notebook cells
import os
from PIL import Image
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

%matplotlib inline
%config InlineBackend.figure_format='retina'

# Import seaborn styles explicitly
import seaborn as sns
# Apply the seaborn style before creating plots
sns.set_style("whitegrid")  # This line sets the Seaborn style

plt.rcParams["figure.figsize"] = (15,10)

In [3]:
# https://play.google.com/store/apps/details?id=com.tranzmate

from google_play_scraper import app, Sort, reviews_all
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

# Memastikan hasil deteksi bahasa konsisten
DetectorFactory.seed = 0

mv_reviews = reviews_all(
    'com.tranzmate',
    sleep_milliseconds=0, # defaults to 0
    lang='id', # defaults to 'en'
    country='id',
    sort=Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
)

In [4]:
# Save moovit reviews into dataframe
df_mvrev = pd.DataFrame(np.array(mv_reviews),columns=['content'])
df_mvrev = df_mvrev.join(pd.DataFrame(df_mvrev.pop('content').tolist()))

df_mvrev.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,48e80d4b-e56b-4800-9236-f00764bc15d2,Danan Jaya,https://play-lh.googleusercontent.com/a-/ALV-U...,Akurat dan sangat diperlukan,5,0,5.177.3.1748,2025-10-15 01:17:07,None,NaT,5.177.3.1748
1,8a745e04-dc85-42ca-8519-b9c1b7ba5c4a,Adellya Zahira,https://play-lh.googleusercontent.com/a-/ALV-U...,"apk gajelas, waktu keberangkatan jak di moovit...",1,0,5.177.3.1748,2025-10-14 12:43:37,None,NaT,5.177.3.1748
2,e271b192-0f8d-43a5-a198-d331798af2a6,ciaa,https://play-lh.googleusercontent.com/a/ACg8oc...,"ga jelas, ga akurat sama sekali sama rute jakl...",1,0,None,2025-10-14 12:38:39,None,NaT,None
3,d8ac4964-a232-4c24-9bb9-accae8372aa4,Rudi Properti,https://play-lh.googleusercontent.com/a-/ALV-U...,gw instal lu ko makin ribet sih anj,1,0,5.177.3.1748,2025-10-13 01:34:34,None,NaT,5.177.3.1748
4,e77bcc6c-959a-45dd-ae99-a56da18245ae,Master Gentak,https://play-lh.googleusercontent.com/a-/ALV-U...,"apk tolol, salah klik iklan blog bgt ni apk",1,0,5.177.0.1745,2025-10-01 12:38:22,None,NaT,5.177.0.1745


In [5]:
df_mvrev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2829 entries, 0 to 2828
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              2829 non-null   object        
 1   userName              2829 non-null   object        
 2   userImage             2829 non-null   object        
 3   content               2829 non-null   object        
 4   score                 2829 non-null   int64         
 5   thumbsUpCount         2829 non-null   int64         
 6   reviewCreatedVersion  1973 non-null   object        
 7   at                    2829 non-null   datetime64[ns]
 8   replyContent          570 non-null    object        
 9   repliedAt             570 non-null    datetime64[ns]
 10  appVersion            1973 non-null   object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 243.2+ KB


In [6]:
# Fungsi untuk mendeteksi bahasa dengan penanganan jika teks terlalu pendek
def detect_language(text):
    try:
        # Pastikan teks adalah string dan tidak kosong
        if isinstance(text, str) and len(text.strip()) > 10:
            return detect(text)
        else:
            return 'unknown'
    except LangDetectException:
        return 'error'

print("Memulai verifikasi bahasa pada kolom 'content'...")
# Terapkan fungsi pada setiap baris di kolom 'content'
df_mvrev['language'] = df_mvrev['content'].apply(detect_language)

print("\nDistribusi bahasa yang terdeteksi:")
print(df_mvrev['language'].value_counts())

# Filter DataFrame untuk hanya menyimpan review Bahasa Indonesia ('id')
df_indonesian = df_mvrev[df_mvrev['language'] == 'id'].copy()
# Hapus kolom 'language' karena tidak diperlukan lagi
df_indonesian.drop(columns=['language'], inplace=True)

print(f"\nJumlah review setelah difilter (hanya Bahasa Indonesia): {len(df_indonesian)} baris.")

Memulai verifikasi bahasa pada kolom 'content'...

Distribusi bahasa yang terdeteksi:
language
id         1852
unknown     703
tl           79
en           57
so           20
tr           19
hr           11
de           11
et           10
ro           10
da            8
lt            7
af            6
cy            4
fi            4
no            4
nl            3
sl            3
hu            3
ca            3
sw            2
lv            2
pt            2
error         1
sv            1
sq            1
ar            1
cs            1
fr            1
Name: count, dtype: int64

Jumlah review setelah difilter (hanya Bahasa Indonesia): 1852 baris.


In [7]:
# Censor usernames
def censor_username(username):
    if len(username) <= 2:
        return username
    else:
        return username[0] + '*' * (len(username) - 2) + username[-1]

df_mvrev['userName'] = df_mvrev['userName'].apply(censor_username)

In [8]:
# Save dataframe to csv
file_name = 'moovit_reviews_raw.csv'
df_mvrev.to_csv(file_name, index=False)

print(f"\n Data mentah berhasil disimpan ke dalam file: '{file_name}'")

df_mvrev.head()


 Data mentah berhasil disimpan ke dalam file: 'moovit_reviews_raw.csv'


,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,language
0,48e80d4b-e56b-4800-9236-f00764bc15d2,D********a,https://play-lh.googleusercontent.com/a-/ALV-U...,Akurat dan sangat diperlukan,5,0,5.177.3.1748,2025-10-15 01:17:07,None,NaT,5.177.3.1748,id
1,8a745e04-dc85-42ca-8519-b9c1b7ba5c4a,A************a,https://play-lh.googleusercontent.com/a-/ALV-U...,"apk gajelas, waktu keberangkatan jak di moovit...",1,0,5.177.3.1748,2025-10-14 12:43:37,None,NaT,5.177.3.1748,id
2,e271b192-0f8d-43a5-a198-d331798af2a6,c**a,https://play-lh.googleusercontent.com/a/ACg8oc...,"ga jelas, ga akurat sama sekali sama rute jakl...",1,0,None,2025-10-14 12:38:39,None,NaT,None,id
3,d8ac4964-a232-4c24-9bb9-accae8372aa4,R***********i,https://play-lh.googleusercontent.com/a-/ALV-U...,gw instal lu ko makin ribet sih anj,1,0,5.177.3.1748,2025-10-13 01:34:34,None,NaT,5.177.3.1748,sl
4,e77bcc6c-959a-45dd-ae99-a56da18245ae,M***********k,https://play-lh.googleusercontent.com/a-/ALV-U...,"apk tolol, salah klik iklan blog bgt ni apk",1,0,5.177.0.1745,2025-10-01 12:38:22,None,NaT,5.177.0.1745,tl
